In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
210,AFG,Asia,Afghanistan,2020-08-07,36937.0,41.0,1298.0,0.0,948.846,1.053,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
362,ALB,Europe,Albania,2020-08-07,6016.0,127.0,188.0,6.0,2090.486,44.131,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
578,DZA,Africa,Algeria,2020-08-07,33626.0,571.0,1273.0,12.0,766.823,13.021,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-06,France,195004.0,0.0,30332.0,0.0,0.0,0.0
0,2020-08-06,China,88426.0,0.0,4685.0,0.0,0.0,0.0
0,2020-08-06,Italy,249101.0,0.0,35208.0,0.0,0.0,0.0
0,2020-08-06,Spain,310555.0,0.0,28527.0,0.0,0.0,0.0
0,2020-08-06,United States,4907853.0,0.0,159579.0,0.0,0.0,0.0
0,2020-08-06,World,19177291.0,0.0,714603.0,0.0,0.0,0.0
0,2020-08-06,United Kingdom,308074.0,0.0,46492.0,0.0,0.0,0.0
0,2020-08-06,Germany,213796.0,0.0,9192.0,0.0,0.0,0.0
0,2020-08-06,Iran,321097.0,0.0,18031.0,0.0,0.0,0.0
0,2020-08-06,Turkey,237092.0,0.0,5800.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-07,France,195004.0,195633.0,30332.0,30312.0,-629.0,20.0
0,2020-08-07,China,88426.0,88460.0,4685.0,4680.0,-34.0,5.0
0,2020-08-07,Italy,249101.0,249204.0,35208.0,35187.0,-103.0,21.0
0,2020-08-06,Spain,310555.0,309855.0,28527.0,28500.0,700.0,27.0
0,2020-08-07,United States,4907853.0,4883646.0,159579.0,160104.0,24207.0,-525.0
0,2020-08-07,World,19177291.0,19076745.0,714603.0,714618.0,100546.0,-15.0
0,2020-08-07,United Kingdom,308074.0,308134.0,46492.0,46413.0,-60.0,79.0
0,2020-08-07,Germany,213796.0,214214.0,9192.0,9183.0,-418.0,9.0
0,2020-08-07,Iran,321097.0,320117.0,18031.0,17816.0,980.0,215.0
0,2020-08-07,Turkey,237092.0,237265.0,5800.0,5798.0,-173.0,2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")